In [1]:
%matplotlib tk
import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d1 import MyEnv
#from test_env_for_lstm import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions

from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.read_wright_weights import save_weights
from utility.read_wright_weights import reload_weights
from utility.save_logs import save_logs
from utility.save_logs import save_hists
from utility.save_logs import save_env_info

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 1
CONTINUAL = True
NUM_EVAL = 10
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

eval_env = MyEnv()
policies_own = {
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
               {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},
               "exploration_config": {"type": "StochasticSampling","random_timesteps":0},"explore":True,}),
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
policies_enem = {
    "red_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
    "red_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space,
              {"model":{"vf_share_layers": False,"use_lstm": True,"max_seq_len": 200},"explore":False,}),
}
# policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config_own = ppo.DEFAULT_CONFIG.copy()
config_own = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 1200*25,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_own,  "policy_mapping_fn": policy_mapping_fn}
         }
config_enem = ppo.DEFAULT_CONFIG.copy()
config_enem = {"env": MyEnv,"num_gpus": 1,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 1,
          "train_batch_size": 600*5*10,
          "batch_mode": "complete_episodes",
          "gamma":0.995, "lr": 2.5e-4,"shuffle_sequences": True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "sgd_minibatch_size": 600, "num_sgd_iter":20,
          "multiagent": {"policies": policies_enem,  "policy_mapping_fn": policy_mapping_fn}
         }

res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config_own,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

adversary = ppo.PPOTrainer(config=config_enem,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    
    #trainer.restore(checkpoint_path=model_path)
    #save_weights("blue_0",trainer)
    #save_weights("blue_1",trainer)

    reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
    reload_weights(policy_id="blue_0",trainer=trainer,set_policy_id="blue_0")
    reload_weights(policy_id="blue_1",trainer=trainer,set_policy_id="blue_1")
    save_weights("red_0",trainer)
    save_weights("red_1",trainer)


models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
hist_dir = os.path.join('./' + PROJECT + '/hist/')
if not os.path.exists(hist_dir):
    os.makedirs(hist_dir)
for j in range(2):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\ale_py\roms\utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):
C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-04-07 00:49:19,897	INFO trainer.py:2141 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-04-07 00:49:19,899	INFO ppo.py:250 -- In multi-agent mode, policies will be optimized sequentially by the multi

2022-04-07 00:49:48,779	INFO trainable.py:130 -- Trainable.setup took 28.885 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-07 00:49:48,781	WARNING util.py:55 -- Install gputil for GPU system monitoring.


2022-04-07 00:50:01,878	INFO trainable.py:130 -- Trainable.setup took 13.049 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-04-07 00:50:01,879	WARNING util.py:55 -- Install gputil for GPU system monitoring.


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [ ]:

eval_env.reset()
save_env_info(eval_env)
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'

best_reward = {"blue_0":0,"blue_1":0}
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.self_play = True
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')
        EVAL_reward = {"blue_0":0,"blue_1":0}
        #check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            #model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
            #trainer.restore(checkpoint_path=model_path)
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            #fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100

            cell_size = 256
            state_0=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_1=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_2=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            state_3=[np.zeros(cell_size, np.float32),np.zeros(cell_size, np.float32)]
            action_dict0 = [0,0]
            action_dict1 = [0,0]
            action_dict2 = [0,0]
            action_dict3 = [0,0]
            rewards = {"blue_0":0,"blue_1":0}
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                #video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                action_dict0 = trainer.compute_single_action(obs["blue_0"],
                                                             state=state_0,prev_action=None,prev_reward=None,
                                                             policy_id="blue_0",explore=False)
                action_dict1 = trainer.compute_single_action(obs["blue_1"],
                                                             state=state_1,prev_action=None,prev_reward=None,
                                                             policy_id="blue_1",explore=False)
                action_dict2 = trainer.compute_single_action(obs["red_0"],
                                                             state=state_2,prev_action=None,prev_reward=None,
                                                             policy_id="red_0",explore=False)
                action_dict3 = trainer.compute_single_action(obs["red_1"],
                                                             state=state_3,prev_action=None,prev_reward=None,
                                                             policy_id="red_1",explore=False)
                
                #action_dict0 = trainer.compute_single_action(obs["blue_0"],policy_id="blue_0")
                #action_dict1 = trainer.compute_single_action(obs["blue_1"],policy_id="blue_1")
                state_0 = action_dict0[1]
                state_1 = action_dict1[1]
                state_2 = action_dict2[1]
                state_3 = action_dict3[1]
                obs, rewards, dones, infos = eval_env.step({"blue_0": action_dict0[0],
                                                            "blue_1": action_dict1[0],
                                                            "red_0": action_dict2[0],
                                                            "red_1": action_dict3[0],})

                env_blue_pos_temp_mod, env_red_pos_temp_mod, env_mrm_pos_temp_mod = render_env.copy_from_env_mod(eval_env)
                if eval_env.timer == 1:
                    env_blue_pos_mod = env_blue_pos_temp_mod
                    env_red_pos_mod = env_red_pos_temp_mod
                    env_mrm_pos_mod = env_mrm_pos_temp_mod
                else:
                    env_blue_pos_mod = np.vstack([env_blue_pos_mod,env_blue_pos_temp_mod])
                    env_red_pos_mod = np.vstack([env_red_pos_mod,env_red_pos_temp_mod])
                    env_mrm_pos_mod = np.vstack([env_mrm_pos_mod,env_mrm_pos_temp_mod])
                EVAL_reward["blue_0"] += rewards["blue_0"]
                EVAL_reward["blue_1"] += rewards["blue_1"]
                # plt.clf()

                # plt.subplots_adjust(left=-0.1,right=1.1,bottom=-0.1,top=1.1)
                # fig.canvas.draw()
                # plt.pause(.01)

                #if record_mode == 0:
                    #img = np.array(fig.canvas.renderer.buffer_rgba())
                    #img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    #video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    save_hists("blue_"+str(i),steps,env_blue_pos_mod,hist_dir)
                    save_hists("red_"+str(i),steps,env_red_pos_mod,hist_dir)
                    save_hists("mrm_"+str(i),steps,env_mrm_pos_mod,hist_dir)
                    # print(f'all done at {env.steps}')
                    break
            if EVAL_reward["blue_0"]> best_reward["blue_0"]:
                save_weights("blue_0",trainer)
                reload_weights(policy_id="red_0",trainer=trainer,set_policy_id="blue_0")
                best_reward["blue_0"] = EVAL_reward["blue_0"]
            if EVAL_reward["blue_1"]> best_reward["blue_1"]:
                save_weights("blue_1",trainer)
                reload_weights(policy_id="red_1",trainer=trainer,set_policy_id="blue_1")
                best_reward["blue_1"] = EVAL_reward["blue_1"]

            
            #if record_mode == 0:
               # video.release()

ray.shutdown()

-------------------------- Scene: 0 --------------------------

----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 0 --------------------------
139 blue_1 DOWN
993 red_0 Shoot at blue_0
1004 red_0 Shoot at blue_0
1009 red_1 Shoot at blue_0
1020 red_1 Shoot at blue_0
1063 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 1063 -0.137729876277264 -50.49046887302631
blue_1 False False 1063 -0.10099940961059735 -1.2030305315428063
-------------------------- Scene: 0 --------------------------
96 blue_0 DOWN
123 blue_1 DOWN
TIME LIMIT LOSE
blue_0 False False 123 -0.10100182800254708 -1.4057956680719084
blue_1 False False 123 -1.1009973613358803 -1.3046351003204857
-------------------------- Scene: 0 --------------------------
286 blue_1 Shoot at red_1 launch distance : 59727.15648536425 True True
297 blue_1 Shoot at red_1 launch distance : 54594.60364631783 True True
339 red_0 Shoo

903 red_0 Shoot at blue_0
907 red_1 Shoot at blue_0
956 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 956 -0.14035539926209917 -27.81297584437308
blue_1 False False 956 -0.10099916592876584 -13.91186228490049
-------------------------- Scene: 0 --------------------------
169 blue_0 DOWN
286 blue_1 Shoot at red_1 launch distance : 59941.69851106115 True True
Same tgt shoot
297 blue_1 Shoot at red_0 launch distance : 53341.94656222876 True True
397 blue_1 Splash :red_0
402 blue_1 Splash :red_1
WIN
blue_0 False True 402 12.885403091251595 11.682153073743095
blue_1 False True 402 28.855552344982936 150.33780873097484
-------------------------- Scene: 0 --------------------------
215 blue_1 DOWN
1072 red_0 Shoot at blue_0
1083 red_0 Shoot at blue_0
1096 red_1 Shoot at blue_0
1107 red_1 Shoot at blue_0
1125 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False False 1125 -0.13639441030852797 -58.83956388873293
blue_1 False False 1125 -0.10100097697519465 -1.3043427350525587
-----------------

TIME LIMIT LOSE
blue_0 False False 1200 0.9939995991934333 57.41493187135638
blue_1 False False 1200 0.8989995991934333 87.19214076258699
-------------------------- Scene: 0 --------------------------
160 blue_0 DOWN
287 blue_1 Shoot at red_0 launch distance : 58515.79836001424 True True
299 blue_1 Shoot at red_0 launch distance : 52252.87871330686 True True
323 red_1 Shoot at blue_1
334 red_1 Shoot at blue_1
388 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 388 -0.10099989022293851 -1.2036646903835808
blue_1 False False 388 0.35867057644372813 83.5051344706254
-------------------------- Scene: 0 --------------------------
296 blue_0 Shoot at red_1 launch distance : 59553.0030224472 True True
Same tgt shoot
306 blue_1 Shoot at red_0 launch distance : 59285.853108421274 True True
Same tgt shoot
309 blue_0 Shoot at red_1 launch distance : 54093.032920847756 True True
363 red_0 Shoot at blue_1
374 red_0 Shoot at blue_1
402 blue_0 Splash :red_1
429 blue_1: Destroyed
576 blue_0 DOWN


2022-04-07 01:55:16,066	WARNING ppo.py:127 -- No data for red_1, not updating kl
2022-04-07 01:55:16,067	WARNING ppo.py:127 -- No data for red_0, not updating kl
2022-04-07 01:55:16,220	WARNING deprecation.py:46 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


agent_timesteps_total: 62220
custom_metrics: {}
date: 2022-04-07_01-55-16
done: false
episode_len_mean: 740.7142857142857
episode_media: {}
episode_reward_max: 244.28145909301588
episode_reward_mean: 86.77617549669972
episode_reward_min: -60.143906623785504
episodes_this_iter: 42
episodes_total: 42
experiment_id: 67406e9c973b4678a42dadfef6f1ea34
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 6.898582935333252
        entropy_coeff: 0.0
        kl: 0.11648424714803696
        model: {}
        policy_loss: 0.06078886240720749
        total_loss: 84.18666076660156
        vf_explained_var: -0.09636684507131577
        vf_loss: 84.10257720947266
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 7.145285129547119
        entropy_coeff: 0.0
        kl: 0.07833901047706604
      

...

287 blue_0 Shoot at red_0 launch distance : 58844.86758368663 True True
Same tgt shoot
299 blue_0 Shoot at red_1 launch distance : 57710.08719199633 True True
402 blue_0 Splash :red_0
410 blue_0 Splash :red_1
WIN
blue_0 False True 410 29.280807017143918 155.2318795586094
blue_1 False True 410 12.92715138055855 10.759878868292681
-------------------------- Scene: 0 --------------------------
188 blue_1 DOWN
368 blue_0 DOWN
TIME LIMIT LOSE
blue_0 False False 368 -1.1030013126729434 -3.5343439226887687
blue_1 False False 368 -0.10100171267294344 -1.3039640057698463
-------------------------- Scene: 0 --------------------------
115 blue_1 DOWN
296 blue_0 Shoot at red_0 launch distance : 58571.79097063543 True True
Same tgt shoot
308 blue_0 Shoot at red_1 launch distance : 59460.37319059354 True True
412 blue_0 Splash :red_0
427 blue_0 Splash :red_1
WIN
blue_0 False True 427 28.931243341758265 121.86940096739278
blue_1 False True 427 12.71063444246084 11.507283160879943
--------------------

307 blue_1 Shoot at red_0 launch distance : 48052.03767427843 True True
Same tgt shoot
Same tgt shoot
340 blue_0 Shoot at red_1 launch distance : 59777.45523926003 True True
345 red_1 Shoot at blue_1
Same tgt shoot
Same tgt shoot
353 blue_0 Shoot at red_1 launch distance : 54262.68173161067 True True
356 red_1 Shoot at blue_1
396 blue_1 Splash :red_0
449 blue_0 DOWN
457 blue_0 Splash :red_1
WIN
blue_0 False True 556 12.057273381294964 132.52580547545202
blue_1 False True 556 12.157273381294965 157.39612440856746
-------------------------- Scene: 0 --------------------------
291 blue_1 Shoot at red_1 launch distance : 57905.130609714586 True True
297 blue_0 Shoot at red_1 launch distance : 52857.741553393345 True True
303 blue_1 Shoot at red_1 launch distance : 53167.65853253008 True True
308 blue_0 Shoot at red_1 launch distance : 48425.06579683433 True True
346 red_0 Shoot at blue_0
357 red_0 Shoot at blue_0
398 blue_1 Splash :red_1
412 blue_0: Destroyed
TIME LIMIT LOSE
blue_0 False F

2022-04-07 08:56:40,691	WARNING ppo.py:127 -- No data for red_1, not updating kl
2022-04-07 08:56:40,691	WARNING ppo.py:127 -- No data for red_0, not updating kl


agent_timesteps_total: 365422
custom_metrics: {}
date: 2022-04-07_08-56-40
done: false
episode_len_mean: 633.87
episode_media: {}
episode_reward_max: 289.921929884021
episode_reward_mean: 101.94915877040357
episode_reward_min: -125.19173855777169
episodes_this_iter: 47
episodes_total: 269
experiment_id: 67406e9c973b4678a42dadfef6f1ea34
hostname: DESKTOP
info:
  learner:
    blue_0:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 7.106440544128418
        entropy_coeff: 0.0
        kl: 0.08672266453504562
        model: {}
        policy_loss: 0.018908565863966942
        total_loss: 111.85910034179688
        vf_explained_var: 0.13868620991706848
        vf_loss: 111.82283782958984
      train: null
    blue_1:
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 0.0002500000118743628
        entropy: 7.42042350769043
        entropy_coeff: 0.0
        kl: 0.07520810514688492
        model: 

285 blue_0 Shoot at red_0 launch distance : 59120.48272167667 True True
288 blue_1 Shoot at red_0 launch distance : 55602.42760321751 True True
296 blue_0 Shoot at red_0 launch distance : 53716.544396753954 True True
303 blue_1 Shoot at red_0 launch distance : 48740.0785613364 True True
338 red_1 Shoot at blue_1
349 red_1 Shoot at blue_1
396 blue_1 Splash :red_0
397 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 1200 0.994008619939092 58.7633679086321
blue_1 False False 1200 0.8990006532724253 90.8314252592898
-------------------------- Scene: 0 --------------------------
245 blue_0 DOWN
288 blue_1 Shoot at red_0 launch distance : 59359.30813328252 True True
302 blue_1 Shoot at red_0 launch distance : 53701.16718084647 True True
339 red_1 Shoot at blue_1
350 red_1 Shoot at blue_1
398 blue_1: Destroyed
TIME LIMIT LOSE
blue_0 False False 398 -0.10099991971150377 -3.5335497476647886
blue_1 False False 398 0.35899281362182955 76.85145209594938
-------------------------- Scene: 0 ----